# 🛠 Setup

In [ ]:
pip install -U "servicefoundry==0.6.6" "mlfoundry==0.6.1"

In [ ]:
import logging
[logging.root.removeHandler(h) for h in logging.root.handlers]
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(name)s] %(levelname)-8s %(message)s')

In [ ]:
!sfy login

In [ ]:
!git clone https://github.com/truefoundry/truefoundry-examples.git
%cd truefoundry-examples
!git checkout demo-add
%cd end-to-end-examples/diabetes-regression

In [ ]:
# Copy workspace FQN from https://app.truefoundry.com/workspaces
from getpass import getpass

WORKSPACE="tfy-cluster-euwe1:demos"
TFY_API_KEY = getpass("Please enter your API Key,( https://app.truefoundry.com/settings ):-")

# ⚡ Training a ML Model using Jobs
In this section we will deploy a training job that trains a SVM model on iris dataset

In [ ]:
! mkdir -p /content/demo/regression/
%cd /content/demo/regression/

In [ ]:
%%writefile train.py
import argparse

import matplotlib.pyplot as plt
import mlfoundry
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import PredictionErrorDisplay


def train(kernel: str):
    X, y = load_diabetes(as_frame=True, return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    regressor = SVR(kernel=kernel)
    model = TransformedTargetRegressor(
        regressor=regressor,
        transformer=QuantileTransformer(n_quantiles=100, output_distribution="normal"),
    ).fit(X_train, y_train)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    PredictionErrorDisplay.from_predictions(
        y_test,
        y_pred,
        kind="actual_vs_predicted",
        scatter_kwargs={"alpha": 0.5},
    )

    run = mlfoundry.get_client().create_run(
        project_name="diabetes-regression", run_name="SVR-with-QT"
    )

    run.log_params(regressor.get_params())
    run.log_metrics({"score": model.score(X_test, y_test)})
    run.log_plots({"actual_vs_predicted": plt})

    model_version = run.log_model(
        name="diabetes-regression",
        model=model,
        framework="sklearn",
        description="SVC model trained on initial data",
        model_schema={
          "features": [
            {"name": c, "type": "float"} for c in X.columns
          ],
          "prediction": "numeric",
        },
        custom_metrics=[{"name": "mean_square_error", "type": "metric", "value_type": "float"}]
    )
    print(f"Logged model: {model_version.fqn}")
    run.end()


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--kernel", default="linear", type=str)
    args = parser.parse_args()

    train(kernel=args.kernel)

In [ ]:
%%writefile requirements.txt
pandas==1.3.5
scikit-learn==1.2.1
mlfoundry==0.6.1
matplotlib==3.2.2

In [ ]:
import logging
from getpass import getpass

from servicefoundry import Build, Job, PythonBuild

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

job = Job(
    name="train",
    image=Build(
        build_spec=PythonBuild(
            command="python train.py",
            requirements_path="requirements.txt",
        )
    ),
    env={
        "TFY_API_KEY": TFY_API_KEY
    }
)
deployment = job.deploy(workspace_fqn=WORKSPACE)

In [ ]:
from servicefoundry.internal.experimental import trigger_job

JOB_DEPLOYMENT_FQN=input("Enter Job Deployment FQN:-")

for kernel in ["linear", "rbf", "poly"]:
    trigger_job(deployment_fqn=JOB_DEPLOYMENT_FQN, command=f"python train.py --kernel {kernel}")

# ☁ Deploy ML Model using a Service (FastAPI)
In this section we will deploy the model trained and logged by our training job as an API

In [ ]:
!mkdir -p /content/demo/regression/
%cd /content/demo/regression/

In [ ]:
%%writefile main.py
import os
from typing import List
import uuid
from datetime import datetime

import mlfoundry as mlf
import pandas as pd
from pydantic import BaseModel
from fastapi import FastAPI

app = FastAPI(docs_url="/")


MODEL_VERSION_FQN = os.environ["MODEL_VERSION_FQN"]
client = mlf.get_client()
model = client.get_model(MODEL_VERSION_FQN).load()


class Instance(BaseModel):
    age: float
    sex: float
    bmi: float
    bp: float
    s1: float
    s2: float
    s3: float
    s4: float
    s5: float
    s6: float


class Request(BaseModel):
    instances: List[Instance]

class Response(BaseModel):
    predictions: List[float]


@app.post("/predict", response_model=Response)
def predict(request: Request):
    features = request.dict()["instances"]

    features_df = pd.DataFrame(features)
    predictions = [float(p) for p in model.predict(features_df)]
    client.log_predictions(
        model_version_fqn=MODEL_VERSION_FQN,
        predictions=[
            mlf.Prediction(
                data_id=uuid.uuid4().hex,
                features=feature,
                prediction_data={
                    "value": prediction,
                },
                occurred_at=datetime.utcnow(),
                raw_data={"data": "any_data"},
            )
            for feature, prediction in zip(features, predictions)
        ],
    )
    return Response(predictions=predictions)

In [ ]:
%%writefile predict_requirements.txt
pandas==1.3.5
scikit-learn==1.2.1
mlfoundry==0.6.1
fastapi==0.81.0
uvicorn==0.18.3

In [ ]:
MODEL_VERSION_FQN = input("Please enter the model version fqn:-")

In [ ]:
import logging
from getpass import getpass
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

from servicefoundry import Build, Service, PythonBuild

service = Service(
    name="deploy",
    image=Build(
        build_spec=PythonBuild(
            command="uvicorn main:app --port 8000 --host 0.0.0.0",
            requirements_path="predict_requirements.txt",
        )
    ),
    ports=[{"port": 8000}],
    env={
        "TFY_API_KEY": TFY_API_KEY,
        "MODEL_VERSION_FQN": MODEL_VERSION_FQN
    },
)
deployment = service.deploy(workspace_fqn=WORKSPACE)

In [ ]:
import mlfoundry

client = mlfoundry.get_client()

client.log_actuals(
    model_version_fqn=MODEL_VERSION_FQN,
    actuals=[mlfoundry.Actual(data_id="88831787f8ac4c3b80ccfc0f709bf143", value=130)],
)
